# Eisen


In [1]:
!pip install --upgrade eisen -q

You should consider upgrading via the '/home/melihozaydin/.virtualenvs/eisen/bin/python -m pip install --upgrade pip' command.


In [1]:
"""
Eisen ABC Challenge starter kit
NOTE: you need to register to the challenge, download and unpack the data in
order to be able to run the following example.
Find more info here: https://abcs.mgh.harvard.edu
This is released under MIT license. Do what you want with this code.
"""
from eisen.datasets import ABCsDataset
from eisen.models.segmentation import VNet
from eisen.io import LoadITKFromFilename
from eisen.transforms import (
    ResampleITKVolumes,
    ITKToNumpy,
    CropCenteredSubVolumes,
    AddChannelDimension,
    MapValues,
    FixedMeanStdNormalization,
    LabelMapToOneHot,
    StackImagesChannelwise,
    FilterFields
)

In [3]:
from eisen.ops.losses import DiceLoss
from eisen.ops.metrics import DiceMetric
from eisen.utils import EisenModuleWrapper
from eisen.utils.workflows import Training

from eisen.utils.logging import LoggingHook
from eisen.utils.logging import TensorboardSummaryHook
from eisen.utils.artifacts import SaveTorchModelHook

from torchvision.transforms import Compose
from torch.utils.data import DataLoader
from torch.optim import Adam

In [4]:
!pwd

/home/melihozaydin/Desktop/Fast-AI/MICCAI 2020 Challenge


In [5]:
# Defining some constants
PATH_DATA = 'Data/ABCs_training_data/'  # path of data as unpacked from the challenge files
PATH_ARTIFACTS = './results'  # path for model results

TASK = 'task1'

NUM_EPOCHS = 100
BATCH_SIZE = 2

VOLUMES_RESOLUTION = [2, 2, 1.5]
VOLUMES_PIXEL_SIZE = [128, 128, 128]


In [6]:
if TASK == 'task1':
    n_out_chan = 5
    label_field = 'label_task1'
else:
    n_out_chan = 10
    label_field = 'label_task2'


In [7]:
"""
Define Readers and Transforms
In order to load data and prepare it for being used by the network, we need to operate 
I/O operations and define transforms to standardize data.
You can add transforms or change the existing ones by editing this
"""

# readers: for images and labels
read_tform = LoadITKFromFilename(['ct', 't1', 't2', label_field], PATH_DATA)

# image manipulation transforms

resample_tform_img = ResampleITKVolumes(
    ['ct', 't1', 't2'],
    VOLUMES_RESOLUTION,
    'linear'
)

resample_tform_lbl = ResampleITKVolumes(
    [label_field],
    VOLUMES_RESOLUTION,
    'nearest'
)

to_numpy_tform = ITKToNumpy(['ct', 't1', 't2', label_field])

crop = CropCenteredSubVolumes(fields=['ct', 't1', 't2', label_field], size=VOLUMES_PIXEL_SIZE)

map_intensities = MapValues(['t1', 't2'], min_value=0.0, max_value=1.0)

normalize_ct = FixedMeanStdNormalization(['ct'], mean=208.0, std=388.0)

if TASK == 'task1':
    map_labels = LabelMapToOneHot([label_field], [1, 2, 3, 4, 5])
else:
    map_labels = LabelMapToOneHot([label_field], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

stack_modalities = StackImagesChannelwise(['ct', 't1', 't2'], 'image')

preserve_only_fields = FilterFields(['image', 'label_task1'])

In [8]:
# create a transform to manipulate and load data
tform = Compose([
    read_tform,
    resample_tform_img,
    resample_tform_lbl,
    to_numpy_tform,
    crop,
    map_intensities,
    normalize_ct,
    map_labels,
    stack_modalities,
    preserve_only_fields
])

In [9]:
# create a dataset from the training set of the ABC dataset
dataset = ABCsDataset(
    PATH_DATA,
    training=True,
    flat_dir_structure=True,  # check documentation
    transform=tform
)
len(dataset)

In [10]:
# Data loader: a pytorch DataLoader is used here to loop through the data as provided by the dataset.
data_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

In [11]:
"""
Building blocks: we define here:
* model
* loss 
* metric
* optimizer 
These components are used during training. 
These blocks will be joined together in a workflow (Eg. training workflow).
"""

# specify model and loss (building blocks)

model = EisenModuleWrapper(
    module=VNet(input_channels=3, output_channels=n_out_chan),
    input_names=['image'],
    output_names=['predictions']
)


In [12]:
# CHANGE TASK HERE if needed!!
loss = EisenModuleWrapper(
    module=DiceLoss(dim=[2, 3, 4]),
    input_names=['predictions', label_field],
    output_names=['dice_loss']
)
# CHANGE TASK HERE if needed!!
metric = EisenModuleWrapper(
    module=DiceMetric(dim=[2, 3, 4]),
    input_names=['predictions', label_field],
    output_names=['dice_metric']
)

In [13]:
optimizer = Adam(model.parameters(), 0.001)

In [14]:
# join all blocks into a workflow (training workflow)
training_workflow = Training(
      model=model,
      losses=[loss],
      data_loader=data_loader,
      optimizer=optimizer,
      metrics=[metric],
      gpu=True
)

In [15]:
from datetime import datetime
now = datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

# create Hook to monitor training and save models
training_loggin_hook = LoggingHook(training_workflow.id, f'Training {date_time}', PATH_ARTIFACTS)

training_summary_hook = TensorboardSummaryHook(training_workflow.id, f'Training {date_time}', PATH_ARTIFACTS)

save_model_hook = SaveTorchModelHook(training_workflow.id, f'Training {date_time}', PATH_ARTIFACTS)

In [16]:
from IPython.display import clear_output

for i in range(NUM_EPOCHS):
    clear_output(wait=True)
    training_workflow.run()

+---------------------+---------------+-----------------+
|        Phase        | dice_loss (L) | dice_metric (M) |
+---------------------+---------------+-----------------+
|  Training - Epoch 0 |   0.9706479   |    0.04478587   |
|  Training - Epoch 1 |   0.9633373   |   0.047551885   |
|  Training - Epoch 2 |   0.9082786   |    0.12253807   |
|  Training - Epoch 3 |   0.7769802   |    0.2326393    |
|  Training - Epoch 4 |   0.6608959   |    0.31124267   |
|  Training - Epoch 5 |   0.57172084  |    0.38438332   |
|  Training - Epoch 6 |   0.51830345  |    0.41532984   |
|  Training - Epoch 7 |   0.4900753   |    0.4256632    |
|  Training - Epoch 8 |   0.44360185  |    0.47072455   |
|  Training - Epoch 9 |   0.4046064   |    0.5364692    |
| Training - Epoch 10 |   0.37272027  |    0.5816662    |
| Training - Epoch 11 |   0.3267279   |    0.63277483   |
| Training - Epoch 12 |   0.30210963  |    0.66029966   |
| Training - Epoch 13 |   0.27489898  |    0.68885386   |
| Training - E

In [56]:
import torch
torch.save(model.state_dict(), 'models/vnet0.877.pt')

# run optimization for NUM_EPOCHS
for i in range(NUM_EPOCHS):
    training_workflow.run()
    
# todo: VALIDATION and INFERENCE code
    +---------------------+---------------+-----------------+
    |        Phase        | dice_loss (L) | dice_metric (M) |
    +---------------------+---------------+-----------------+
    |  Training - Epoch 0 |   0.96793664  |   0.043706942   |
    |  Training - Epoch 1 |   0.89846253  |    0.18680497   |
    |  Training - Epoch 2 |   0.7522728   |    0.30640912   |
    |  Training - Epoch 3 |   0.6449693   |    0.33607706   |
    |  Training - Epoch 4 |   0.5778612   |    0.36931768   |
    |  Training - Epoch 5 |   0.52831995  |    0.39147386   |
    |  Training - Epoch 6 |   0.47558337  |    0.48128268   |
    |  Training - Epoch 7 |   0.42666447  |    0.52758354   |
    |  Training - Epoch 8 |    0.349747   |    0.62017804   |
    |  Training - Epoch 9 |   0.2901562   |    0.6782357    |
    | Training - Epoch 10 |   0.27295968  |    0.6923639    |
    | Training - Epoch 11 |   0.25372407  |    0.71003294   |
    | Training - Epoch 12 |   0.24238265  |    0.7218047    |
    | Training - Epoch 13 |   0.23098417  |    0.73275656   |
    | Training - Epoch 14 |   0.22004963  |    0.74475867   |
    | Training - Epoch 15 |   0.22097065  |    0.74230677   |
    | Training - Epoch 16 |   0.21581535  |    0.7478182    |
    | Training - Epoch 17 |   0.20848322  |    0.75519836   |
    | Training - Epoch 18 |   0.2041212   |    0.7594452    |
    | Training - Epoch 19 |   0.20031235  |    0.7636146    |
    | Training - Epoch 20 |   0.19871724  |    0.76559794   |
    | Training - Epoch 21 |   0.19231819  |    0.7721731    |
    | Training - Epoch 22 |   0.18916644  |    0.7761358    |
    | Training - Epoch 23 |   0.18533538  |    0.77972853   |
    | Training - Epoch 24 |   0.18625353  |    0.7784818    |
    | Training - Epoch 25 |   0.18426937  |    0.78109103   |
    | Training - Epoch 26 |   0.17872038  |    0.78699726   |
    | Training - Epoch 27 |   0.17534561  |    0.79003257   |
    | Training - Epoch 28 |   0.17263515  |    0.79372025   |
    | Training - Epoch 29 |   0.17081174  |     0.794877    |
    | Training - Epoch 30 |   0.16819598  |    0.79776156   |
    | Training - Epoch 31 |   0.16726722  |    0.79900014   |
    | Training - Epoch 32 |   0.16900492  |    0.7970732    |
    | Training - Epoch 33 |   0.16413586  |    0.80289084   |
    | Training - Epoch 34 |   0.16254856  |    0.8038067    |
    | Training - Epoch 35 |   0.16372152  |    0.8032655    |
    | Training - Epoch 36 |   0.15768503  |    0.8095241    |
    | Training - Epoch 37 |   0.15977882  |    0.80724674   |
    | Training - Epoch 38 |   0.16016683  |    0.8070099    |
    | Training - Epoch 39 |   0.15268648  |    0.81475824   |
    | Training - Epoch 40 |   0.15210675  |    0.81605357   |
    | Training - Epoch 41 |   0.15383334  |    0.81352913   |
    | Training - Epoch 42 |   0.15196165  |    0.81570715   |
    | Training - Epoch 43 |   0.14960389  |    0.81841767   |
    | Training - Epoch 44 |   0.14609665  |    0.82224846   |
    | Training - Epoch 45 |   0.14577235  |    0.8224502    |
    | Training - Epoch 46 |   0.14366336  |    0.8247403    |
    | Training - Epoch 47 |   0.14182788  |    0.8267273    |
    | Training - Epoch 48 |   0.1424644   |    0.8266169    |
    | Training - Epoch 49 |   0.14381047  |     0.82458     |
    | Training - Epoch 50 |   0.14254089  |    0.8261815    |
    | Training - Epoch 51 |   0.13822462  |    0.83112335   |
    | Training - Epoch 52 |   0.13436086  |    0.8354083    |
    | Training - Epoch 53 |   0.13338886  |    0.8366843    |
    | Training - Epoch 54 |   0.13237219  |    0.83773416   |
    | Training - Epoch 55 |   0.1423021   |    0.82635903   |
    | Training - Epoch 56 |   0.13526128  |    0.83461714   |
    | Training - Epoch 57 |   0.12947176  |    0.84092456   |
    | Training - Epoch 58 |    0.129506   |    0.84092146   |
    | Training - Epoch 59 |   0.12563513  |    0.84576124   |
    | Training - Epoch 60 |   0.12391328  |    0.8474134    |
    | Training - Epoch 61 |  0.123863295  |    0.84764105   |
    | Training - Epoch 62 |   0.12486937  |    0.8466209    |
    | Training - Epoch 63 |   0.12470291  |    0.84678817   |
    | Training - Epoch 64 |  0.123977326  |    0.8473742    |
    | Training - Epoch 65 |   0.11795328  |    0.85486525   |
    | Training - Epoch 66 |   0.11677351  |    0.85593075   |
    | Training - Epoch 67 |   0.11396389  |    0.85919213   |
    | Training - Epoch 68 |   0.11319907  |    0.8603511    |
    | Training - Epoch 69 |   0.11129231  |    0.86245924   |
    | Training - Epoch 70 |   0.1112473   |    0.86251634   |
    | Training - Epoch 71 |   0.1094593   |    0.86497283   |
    | Training - Epoch 72 |   0.10717194  |    0.86751705   |
    | Training - Epoch 73 |   0.10481413  |    0.87010956   |
    | Training - Epoch 74 |   0.10569673  |     0.869493    |
    | Training - Epoch 75 |  0.106171586  |    0.8688619    |
    | Training - Epoch 76 |   0.10719231  |    0.86747974   |
    | Training - Epoch 77 |  0.103855126  |    0.8712129    |
    | Training - Epoch 78 |  0.099139936  |    0.8771581    |
    | Training - Epoch 79 |   0.09859477  |    0.8775554    |
    | Training - Epoch 80 |   0.09779638  |    0.8789633    |
    | Training - Epoch 81 |  0.097967975  |    0.87831175   |
    | Training - Epoch 82 |   0.09648657  |    0.8802494    |
    | Training - Epoch 83 |   0.0988178   |    0.8773187    |
    | Training - Epoch 84 |   0.09536934  |    0.8811702    |
    | Training - Epoch 85 |   0.09503206  |    0.8820635    |
    | Training - Epoch 86 |   0.09292293  |    0.8843886    |
    | Training - Epoch 87 |   0.09223585  |    0.8848819    |
    | Training - Epoch 88 |   0.08824333  |    0.8902988    |
    | Training - Epoch 89 |   0.08673954  |    0.8920021    |
    | Training - Epoch 90 |  0.084142536  |    0.8951429    |
    | Training - Epoch 91 |  0.080945194  |    0.89951605   |
    | Training - Epoch 92 |   0.08215775  |    0.8976262    |
    | Training - Epoch 93 |   0.0829654   |    0.8966049    |
    | Training - Epoch 94 |   0.08655508  |    0.8920223    |
    | Training - Epoch 95 |   0.08357894  |    0.89558625   |
    | Training - Epoch 96 |   0.08275148  |    0.8968379    |
    | Training - Epoch 97 |   0.07770972  |    0.90333366   |
    | Training - Epoch 98 |   0.07604958  |    0.90533125   |
    | Training - Epoch 99 |   0.0741501   |    0.9076199    |
    +---------------------+---------------+-----------------+